In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('tree_ca').getOrCreate()

In [0]:
from pyspark.ml.classification import RandomForestClassifier, GBTClassifier, DecisionTreeClassifier
from pyspark.ml.feature import VectorAssembler, VectorIndexer, OneHotEncoder, StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.sql.functions import year, month, dayofmonth

In [0]:
df = sqlContext.sql("SELECT * FROM college_csv")
df.describe().show()

+-------+--------------------+-------+------------------+------------------+----------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+----------------+------------------+
summary| School|Private| Apps| Accept| Enroll| Top10perc| Top25perc| F_Undergrad| P_Undergrad| Outstate| Room_Board| Books| Personal| PhD| Terminal| S_F_Ratio| perc_alumni| Expend| Grad_Rate|
+-------+--------------------+-------+------------------+------------------+----------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+----------------+------------------+
 count| 777| 777| 777| 777| 777| 777| 777| 777| 777| 777| 777| 777| 777| 777| 777| 777| 777| 777| 777|
 mean| null| null|3001.6383526383524|2018.8043758043757|779.972972972973| 27.55855855855856| 55.7966537966538|3699.907335907336|855.2985842985843| 10440.66924066924| 4357.526383526383| 549.3809523809524|1340.6422136422136| 72.66023166023166| 79.70270270270271|14.089703989703986|22.743886743886744|9660.17117117117| 65.46332046332046|
 stddev| null| null|3870.2014844352884| 2451.11397099263| 929.17619013287|17.640364385452134|19.804777595131373|4850.420530887386|1522.431887295513|4023.0164841119727|1096.6964155935289|165.10536013709253| 677.071453590578|16.328154687939314|14.722358527903374|3.9583491352055478| 12.39180148937615|5221.76843985609|17.177709897155403|
 min|Abilene Christian...| No| 81| 72| 35| 1| 9| 139| 1| 2340| 1780| 96| 250| 8| 24| 2.5| 0| 3186| 10|
 max|York College of P...| Yes| 48094| 26330| 6392| 96| 100| 31643| 21836| 21700| 8124| 2340| 6800| 103| 100| 39.8| 64| 56233| 118|
+-------+--------------------+-------+------------------+------------------+----------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+----------------+------------------+

In [0]:
df.columns

Out[4]: ['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [0]:
ass = VectorAssembler(inputCols=['Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate'], outputCol='features')

In [0]:
op = ass.transform(df)

In [0]:
indxr = StringIndexer(inputCol = 'Private', outputCol='PrivateIndex')

In [0]:
opFixed = indxr.fit(op).transform(op)

In [0]:
opFixed.printSchema()

root
-- School: string (nullable = true)
-- Private: string (nullable = true)
-- Apps: integer (nullable = true)
-- Accept: integer (nullable = true)
-- Enroll: integer (nullable = true)
-- Top10perc: integer (nullable = true)
-- Top25perc: integer (nullable = true)
-- F_Undergrad: integer (nullable = true)
-- P_Undergrad: integer (nullable = true)
-- Outstate: integer (nullable = true)
-- Room_Board: integer (nullable = true)
-- Books: integer (nullable = true)
-- Personal: integer (nullable = true)
-- PhD: integer (nullable = true)
-- Terminal: integer (nullable = true)
-- S_F_Ratio: double (nullable = true)
-- perc_alumni: integer (nullable = true)
-- Expend: integer (nullable = true)
-- Grad_Rate: integer (nullable = true)
-- features: vector (nullable = true)
-- PrivateIndex: double (nullable = false)

In [0]:
fdf = opFixed.select('features','PrivateIndex')
fdf.show()

+--------------------+------------+
 features|PrivateIndex|
+--------------------+------------+
[1660.0,1232.0,72...| 0.0|
[2186.0,1924.0,51...| 0.0|
[1428.0,1097.0,33...| 0.0|
[417.0,349.0,137....| 0.0|
[193.0,146.0,55.0...| 0.0|
[587.0,479.0,158....| 0.0|
[353.0,340.0,103....| 0.0|
[1899.0,1720.0,48...| 0.0|
[1038.0,839.0,227...| 0.0|
[582.0,498.0,172....| 0.0|
[1732.0,1425.0,47...| 0.0|
[2652.0,1900.0,48...| 0.0|
[1179.0,780.0,290...| 0.0|
[1267.0,1080.0,38...| 0.0|
[494.0,313.0,157....| 0.0|
[1420.0,1093.0,22...| 0.0|
[4302.0,992.0,418...| 0.0|
[1216.0,908.0,423...| 0.0|
[1130.0,704.0,322...| 0.0|
[3540.0,2001.0,10...| 1.0|
+--------------------+------------+
only showing top 20 rows

In [0]:
train, test = fdf.randomSplit([.7,.3])

In [0]:
dtc = DecisionTreeClassifier(labelCol='PrivateIndex')
rfc = RandomForestClassifier(labelCol='PrivateIndex', numTrees=100)
gbt = GBTClassifier(labelCol='PrivateIndex')

In [0]:
dtcModel = dtc.fit(train)
rfcModel = rfc.fit(train)
gbtModel = gbt.fit(train)


In [0]:
dtcPreds = dtcModel.transform(test)
rfcPreds = rfcModel.transform(test)
gbtPreds = gbtModel.transform(test)


In [0]:
binEval=BinaryClassificationEvaluator(labelCol='PrivateIndex')

In [0]:
print('DTC')
print(binEval.evaluate(dtcPreds))

DTC
0.901454033771107

In [0]:
print('RFC')
print(binEval.evaluate(rfcPreds))

RFC
0.9813320825515949

In [0]:
binEval2=BinaryClassificationEvaluator(labelCol='PrivateIndex', rawPredictionCol='prediction')

In [0]:
print('GBT')
print(binEval2.evaluate(gbtPreds))

GBT
0.8834427767354598

In [0]:
acc = MulticlassClassificationEvaluator(labelCol='PrivateIndex', metricName='accuracy')

In [0]:
dtcAcc = acc.evaluate(dtcPreds)
rfcAcc = acc.evaluate(rfcPreds)
gbtAcc = acc.evaluate(gbtPreds)


In [0]:
dtcAcc

Out[35]: 0.8995633187772926

In [0]:
rfcAcc

Out[36]: 0.9301310043668122

In [0]:
gbtAcc

Out[37]: 0.8995633187772926